In [ ]:
## 2. Setup

import math
import random
import time
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from tqdm.notebook import tqdm

# Set random seed
SEED = 2025
random.seed(SEED)
np.random.seed(SEED)

# High precision for coordinates
getcontext().prec = 50

print("Libraries imported and seed set.")

In [ ]:
# SA parameter normalization helper
def normalize_sa_params(params: dict):
    """Return canonical SA parameters with robust fallbacks.
    Supports multiple naming conventions to avoid KeyError.
    Keys mapped: T0, Tmin, move_scale, rot_scale, compression, iterations.
    """
    if params is None:
        params = {}
    T0 = (params.get('initial_temp') or params.get('T0') or params.get('start_temp') or 1.0)
    Tmin = (params.get('min_temp') or params.get('Tmin') or params.get('final_temp') or params.get('end_temp') or 0.001)
    move_scale = (params.get('move_scale') or params.get('move') or 0.5)
    rot_scale = (params.get('rot_scale') or params.get('rotation_scale') or params.get('rotate_scale') or 30.0)
    compression = (params.get('compression') or params.get('compress') or 0.05)
    iterations = (params.get('iterations') or params.get('iters') or params.get('n_iterations') or 50000)
    base_seed = params.get('base_seed', 2025)
    return {
        'T0': float(T0),
        'Tmin': float(Tmin),
        'move_scale': float(move_scale),
        'rot_scale': float(rot_scale),
        'compression': float(compression),
        'iterations': int(iterations),
        'base_seed': int(base_seed)
    }


In [ ]:
# Configuration for Advanced Features
CONFIG = {
    # Hybrid Search Approach
    'hybrid_search': {
        'enabled': True,
        'switch_n': 25,  # Use forward building for N <= 25
    },
    
    # Dynamic Beam Width
    'dynamic_beam_width': {
        'enabled': True,
        'base_width': 12,
        'critical_n_multiplier': 2.0, # Wider for critical sizes
    },
    
    # Beam Search Diversity
    'beam_diversity': {
        'enabled': True,
        'max_children_per_parent': 3, # Prevent one parent from dominating the next generation
    },
    
    # Multi-Start Optimization
    'multi_start': {
        'enabled': True,
        'n_starts': 3,
    },
    
    # Optimization Function Enhancements
    'optimization': {
        'adaptive_temperature': True,
        'smart_move_selection': True, # Bias moves toward boundary
        'batch_moves': False, # Try moving multiple trees (complex)
        'boundary_bias_strength': 0.7, # 70% chance to pick boundary tree
        'reheating': True, # Restart SA if stuck
        'compaction_pass': True, # Pure translation pass to close gaps
    },
    
    # Post-Processing & Polishing
    'post_processing': {
        'polishing': True, # Run a low-temp refinement phase at the end
        'polishing_iterations': 2000,
    },
    
    # Geometric & Heuristic Improvements
    'heuristics': {
        'corner_filling': True,
        'symmetry_breaking': True,
        'boundary_alignment': True,
    },
    
    # Validation
    'validation': {
        'strict_boundary': True,
        'collision_check': True,
    },
    
    # Debugging
    'debug': {
        'verbose_logging': True,
        'plot_convergence': True,
    }
}

# Bucketed SA Parameters
SA_PARAMS = {
    'large': { # N >= 150
        'iterations': 1000000, # 1M+
        'step_size': 0.5,
        'angle_step': 5.0, # Restricted rotation
        'initial_temp': 1.0,
        'final_temp': 1e-6,
        'compression': 0.2, # Strong compression
    },
    'medium': { # 50 <= N < 150
        'iterations': 500000,
        'step_size': 0.8,
        'angle_step': 15.0,
        'initial_temp': 1.5,
        'final_temp': 1e-6,
        'compression': 0.1,
    },
    'small': { # N < 50
        'iterations': 200000,
        'step_size': 1.0,
        'angle_step': 360.0, # Full rotation
        'initial_temp': 2.0,
        'final_temp': 1e-6,
        'compression': 0.05,
    }
}

def get_sa_params(n):
    if n >= 150:
        return SA_PARAMS['large']
    elif n >= 50:
        return SA_PARAMS['medium']
    else:
        return SA_PARAMS['small']

print("Configuration loaded:", CONFIG)
print("SA Parameters loaded.")

In [ ]:
## 3. Load Data

# Load the sample submission to understand the required output format
try:
    sample_sub = pd.read_csv('test.csv')
    print("Sample Submission Shape:", sample_sub.shape)
    print(sample_sub.head())
except FileNotFoundError:
    print("Sample submission not found, proceeding without it.")

## 4. Baseline Strategy

We define the `GreedyPacker` class here. It encapsulates the logic for placing a single tree into an existing configuration.

In [ ]:
class GreedyPacker:
    def __init__(self, n_trials=100, step_size=0.2, fine_step=0.02):
        self.n_trials = n_trials
        self.step_size = step_size
        self.fine_step = fine_step

    def _generate_weighted_angle(self):
        """
        Generates a random angle with a distribution weighted by abs(sin(2*angle)).
        This helps place more trees in corners (diagonals).
        """
        if not CONFIG['heuristics']['corner_filling']:
            return random.uniform(0, 2 * math.pi)
            
        while True:
            angle = random.uniform(0, 2 * math.pi)
            if random.uniform(0, 1) < abs(math.sin(2 * angle)):
                return angle

    def place_next_tree(self, existing_trees, tree_class):
        """Finds the best position for the next tree given existing trees."""
        if not existing_trees:
            return tree_class(0, 0, 0)

        existing_polys = [t.polygon for t in existing_trees]
        tree_index = STRtree(existing_polys)
        
        # Calculate current bounds and center
        minx, miny, maxx, maxy = unary_union(existing_polys).bounds
        center_x = (minx + maxx) / 2
        center_y = (miny + maxy) / 2
        
        best_tree = None
        min_metric = float('inf')

        for _ in range(self.n_trials):
            # Random angle for the tree itself
            angle = random.uniform(0, 360)
            
            # Weighted approach angle (bias towards diagonals)
            approach_angle = self._generate_weighted_angle()
            vx, vy = math.cos(approach_angle), math.sin(approach_angle)
            
            # Start far away
            radius = max(maxx - minx, maxy - miny) + 10.0
            candidate = tree_class(0, 0, angle)
            
            # Move in
            current_r = radius
            collision = False
            
            # Coarse search
            while current_r > 0:
                px, py = center_x + current_r * vx, center_y + current_r * vy
                candidate.update_position(px, py, angle)
                
                query_indices = tree_index.query(candidate.polygon)
                if any(candidate.polygon.intersects(existing_polys[i]) for i in query_indices):
                    collision = True
                    break
                current_r -= self.step_size
            
            # Fine tune
            if collision:
                current_r += self.step_size
                while True:
                    current_r -= self.fine_step
                    px, py = center_x + current_r * vx, center_y + current_r * vy
                    candidate.update_position(px, py, angle)
                    
                    query_indices = tree_index.query(candidate.polygon)
                    if any(candidate.polygon.intersects(existing_polys[i]) for i in query_indices):
                        # Collision found, step back once and stop
                        current_r += self.fine_step
                        px, py = center_x + current_r * vx, center_y + current_r * vy
                        candidate.update_position(px, py, angle)
                        break
            else:
                candidate.update_position(center_x, center_y, angle)

            # Metric: Minimize the side length of the new bounding box
            t_minx, t_miny, t_maxx, t_maxy = candidate.polygon.bounds
            new_minx = min(minx, t_minx)
            new_miny = min(miny, t_miny)
            new_maxx = max(maxx, t_maxx)
            new_maxy = max(maxy, t_maxy)
            
            new_side = max(new_maxx - new_minx, new_maxy - new_miny)
            
            # Tie-breaker: distance to center
            dist_sq = (px - center_x)**2 + (py - center_y)**2
            
            metric = new_side + (dist_sq * 1e-6)
            
            if metric < min_metric:
                min_metric = metric
                best_tree = tree_class(px, py, angle)
                
        return best_tree

## 5. Feature Engineering Module

Here we define the geometric features of the problem: the `ChristmasTree` class and helper functions for bounding boxes.

In [ ]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree."""
    def __init__(self, center_x=0, center_y=0, angle=0):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self.polygon = self._create_polygon()

    def _create_polygon(self):
        # Tree dimensions
        coords = [
            (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
            (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
            (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
        ]
        poly = Polygon(coords)
        rotated = affinity.rotate(poly, self.angle, origin=(0, 0))
        return affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

    def update_position(self, x, y, angle):
        self.center_x = x
        self.center_y = y
        self.angle = angle
        self.polygon = self._create_polygon()

def get_bounds(trees):
    if not trees: return 0
    minx, miny, maxx, maxy = unary_union([t.polygon for t in trees]).bounds
    return max(maxx - minx, maxy - miny)

In [ ]:
# --- NUMBA ACCELERATED GEOMETRY KERNEL ---
# Refactored to optimization_utils.py for performance and caching
import importlib
import optimization_utils
importlib.reload(optimization_utils)

from optimization_utils import (
    get_poly, get_bbox, pip, seg_intersect, overlap, 
    check_overlap_single_cached, check_overlap_pair_cached, 
    calc_side, calc_side_cached, get_global_bbox, get_global_bbox_cached, 
    find_corner_trees, find_corner_trees_cached, sa_numba
)

# get_bounds is defined in the previous cell (Cell 7) using Shapely.
# calc_side is the Numba equivalent for the optimization loop.

print("Imported optimized Numba kernel from optimization_utils.py")

In [ ]:
# Load and evaluate the existing test.csv to verify the score
import pandas as pd
try:
    test_df = pd.read_csv('test.csv')
    print("Loaded test.csv")
    
    # Parse the 's' prefix
    def parse_s(val):
        return float(str(val).replace('s', ''))
    
    test_df['x'] = test_df['x'].apply(parse_s)
    test_df['y'] = test_df['y'].apply(parse_s)
    test_df['deg'] = test_df['deg'].apply(parse_s)
    
    # Reconstruct trees and calculate score
    total_test_score = 0
    for n in range(1, 201):
        # Get rows for this N
        # The ID format is N_i, e.g., 001_0
        # We need to filter by the prefix
        prefix = f"{n:03d}_"
        rows = test_df[test_df['id'].str.startswith(prefix)]
        
        if len(rows) != n:
            print(f"Warning: N={n} has {len(rows)} rows, expected {n}")
            continue
            
        trees = []
        for _, row in rows.iterrows():
            trees.append(ChristmasTree(row['x'], row['y'], row['deg']))
            
        side = get_bounds(trees)
        score_n = (side ** 2) / n
        total_test_score += score_n
        
    print(f"Score of test.csv: {total_test_score:.4f}")
    
except Exception as e:
    print(f"Could not evaluate test.csv: {e}")


In [ ]:
# Diagnostic: Compare all_solutions dict vs submission_rows scoring
# This helps identify why scores might differ

if 'all_solutions' in globals() and 'submission_rows' in globals():
    print("Comparing scoring methods...\n")
    
    # Method 1: all_solutions dict
    score_method1 = 0
    for n, trees in all_solutions.items():
        side = get_bounds(trees)
        score_method1 += (side ** 2) / n
    
    # Method 2: submission_rows
    score_method2 = 0
    configs = {}
    for row in submission_rows:
        n = int(row[0].split('_')[0])
        if n not in configs:
            configs[n] = []
        x = float(row[1].replace('s', ''))
        y = float(row[2].replace('s', ''))
        deg = float(row[3].replace('s', ''))
        configs[n].append(ChristmasTree(x, y, deg))
    
    for n, trees in configs.items():
        side = get_bounds(trees)
        score_method2 += (side ** 2) / n
    
    print(f"Method 1 (all_solutions dict): {score_method1:.6f} ({len(all_solutions)} configs)")
    print(f"Method 2 (submission_rows):    {score_method2:.6f} ({len(configs)} configs)")
    print(f"Difference: {abs(score_method1 - score_method2):.6f}")
    
    if len(all_solutions) != len(configs):
        print(f"\n⚠️ WARNING: Config count mismatch!")
        missing_in_dict = set(configs.keys()) - set(all_solutions.keys())
        missing_in_rows = set(all_solutions.keys()) - set(configs.keys())
        if missing_in_dict:
            print(f"  Missing in all_solutions: {sorted(missing_in_dict)}")
        if missing_in_rows:
            print(f"  Missing in submission_rows: {sorted(missing_in_rows)}")
    else:
        print(f"\n✓ Both methods have same number of configurations")
else:
    print("Run the main processing cell first to populate variables")


## 6. Model Training (Optional)

For this geometric packing problem, standard supervised learning is less applicable than search algorithms. However, one could train a model to predict the optimal *order* of placement or the optimal *angle* given the current boundary shape. We skip this for the baseline.

In [ ]:
# Placeholder for ML model training
# model = LGBMRegressor(...)
# model.fit(X_train, y_train)

## 7. Optimization Strategy

We define a modular optimization function. Currently, it's a placeholder for a more advanced local search (e.g., trying to wiggle trees after placement).

In [ ]:
def center_packing(trees):
    """Centers the packing at (0,0) based on the bounding box center."""
    if not trees: return trees
    minx, miny, maxx, maxy = unary_union([t.polygon for t in trees]).bounds
    cx = (minx + maxx) / 2
    cy = (miny + maxy) / 2
    
    # If already centered (small epsilon), skip
    if abs(cx) < 1e-9 and abs(cy) < 1e-9:
        return trees
        
    for t in trees:
        t.update_position(t.center_x - cx, t.center_y - cy, t.angle)
    return trees

def optimize_packing(trees, params, target_side=None):
    """
    Simulated Annealing with Numba Acceleration.
    OPTIMIZED: Using collision_scale=1.0 for accurate and faster collision detection.
    """
    if not trees: return trees
    
    n = len(trees)
    
    # Extract data for Numba
    xs = np.array([t.center_x for t in trees], dtype=np.float64)
    ys = np.array([t.center_y for t in trees], dtype=np.float64)
    angs = np.array([t.angle for t in trees], dtype=np.float64)
    
    # Parameters
    iterations = params.get('iterations', 10000)
    T0 = params.get('initial_temp', 1.0)
    Tmin = params.get('final_temp', 1e-6)
    move_scale = params.get('step_size', 0.5)
    rot_scale = params.get('angle_step', 10.0)
    compression = params.get('compression', 0.0)
    
    # Generate a random seed for this run
    seed = np.random.randint(0, 1000000)
    
    # Handle target_side default for Numba
    ts = target_side if target_side is not None else 0.0
    
    # Run Numba SA with collision_scale=1.0 (exact collision detection)
    # This is faster and more accurate than using a buffer
    best_xs, best_ys, best_angs, best_side = sa_numba(
        xs, ys, angs, n, iterations, T0, Tmin, move_scale, rot_scale, seed, compression, 
        collision_scale=1.0, target_side=ts
    )
    
    # Update trees
    for i in range(n):
        trees[i].update_position(best_xs[i], best_ys[i], best_angs[i])
        
    return center_packing(trees)


In [ ]:
def generate_lattice(n=200):
    """
    Generates a structured lattice packing for N=200.
    Uses a brick/triangular layout with alternating orientations.
    """
    print(f"Generating Lattice Initialization for N={n}...")
    
    # Approximate dimensions of the tree bounding box
    # Tree is roughly 0.5 wide and 1.0 tall (centered at 0,0)
    # But effective packing width/height is smaller due to shape
    dx = 0.25  # Horizontal spacing
    dy = 0.45  # Vertical spacing
    
    # Calculate grid size
    # We want a roughly square aspect ratio
    cols = int(math.sqrt(n * (dy/dx)))
    rows = math.ceil(n / cols)
    
    trees = []
    count = 0
    
    # Center the grid
    start_x = -((cols - 1) * dx) / 2
    start_y = -((rows - 1) * dy) / 2
    
    for r in range(rows):
        for c in range(cols):
            if count >= n: break
            
            x = start_x + c * dx
            y = start_y + r * dy
            
            # Offset every other row for triangular/brick pattern
            if r % 2 == 1:
                x += dx / 2
                
            # Alternating orientation (Checkerboard or Row-based)
            # Row-based alternation:
            angle = 0 if r % 2 == 0 else 180
            
            # Checkerboard alternation (optional, uncomment to try):
            # angle = 0 if (r + c) % 2 == 0 else 180
            
            trees.append(ChristmasTree(x, y, angle))
            count += 1
            
    return center_packing(trees)

## 8. Submission Generation

This section runs the full pipeline and generates the submission file.

In [66]:
from joblib import Parallel, delayed
import multiprocessing
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import os
import datetime

# Load baseline from test.csv
known_solutions = {}
try:
    print("Loading baseline from test.csv...")
    baseline_df = pd.read_csv('test.csv')
    
    def parse_s(val):
        return float(str(val).replace('s', ''))
    
    baseline_df['x'] = baseline_df['x'].apply(parse_s)
    baseline_df['y'] = baseline_df['y'].apply(parse_s)
    baseline_df['deg'] = baseline_df['deg'].apply(parse_s)
    
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        rows = baseline_df[baseline_df['id'].str.startswith(prefix)]
        if len(rows) == n:
            trees = []
            for _, row in rows.iterrows():
                trees.append(ChristmasTree(row['x'], row['y'], row['deg']))
            known_solutions[n] = trees
            
    print(f"Loaded {len(known_solutions)} configurations from test.csv")
    
except Exception as e:
    print(f"Could not load baseline: {e}")

# Helper function for parallel execution (REVERSE MODE) - OPTIMIZED
def process_beam_candidate_reverse(base_trees, n, packer_params, target_side=None, remove_idx=None, parent_id=None, task_id=0):
    """
    Takes a solution of size >= n.
    If size > n, removes trees to reach size n.
    Then optimizes.
    OPTIMIZED: Reduced tree copying and faster gravity passes.
    """
    # Re-seed random for this process
    seed_val = (int(time.time() * 1000000) + os.getpid() + task_id) % (2**32)
    random.seed(seed_val)
    np.random.seed(seed_val)
    
    # Shallow copy first - only deep copy if we need to modify
    current_trees = list(base_trees)
    
    # Remove trees if needed (Shrink strategy)
    # Only create new objects when we actually remove trees
    needs_copy = len(current_trees) > n
    if needs_copy:
        current_trees = [ChristmasTree(t.center_x, t.center_y, t.angle) for t in current_trees]
        
    removal_count = 0
    while len(current_trees) > n:
        if remove_idx is not None and remove_idx < len(current_trees):
            # Remove specific tree requested
            current_trees.pop(remove_idx)
            remove_idx = None # Only use once
        else:
            # Fallback: Remove a tree that contributes to the bounds (Outliers)
            # Calculate global bounds
            union = unary_union([t.polygon for t in current_trees])
            minx, miny, maxx, maxy = union.bounds
            
            candidates_to_remove = []
            for i, t in enumerate(current_trees):
                tb = t.polygon.bounds
                # Check if touching global bounds (Contribution to Envelope)
                if (abs(tb[0] - minx) < 1e-3 or abs(tb[1] - miny) < 1e-3 or 
                    abs(tb[2] - maxx) < 1e-3 or abs(tb[3] - maxy) < 1e-3):
                    candidates_to_remove.append(i)
            
            if candidates_to_remove:
                idx = random.choice(candidates_to_remove)
            else:
                idx = random.randint(0, len(current_trees) - 1)
            current_trees.pop(idx)
            removal_count += 1
            
            # Gravity Pass after removal - less frequent and shorter
            # Only every 10 removals instead of every 5
            if removal_count % 10 == 0:
                current_trees = optimize_packing(current_trees, {
                    'iterations': 2000,  # Reduced from 3000
                    'step_size': 0.5,
                    'angle_step': 5.0,
                    'initial_temp': 0.1,
                    'compression': 0.5
                })
            
    # ADAPTIVE OPTIMIZATION based on N (Bucketed)
    params = get_sa_params(n)
    
    candidate_trees = optimize_packing(current_trees, params, target_side=target_side)
        
    # Score Candidate
    side_candidate = get_bounds(candidate_trees)
    score_candidate = (side_candidate ** 2) / n
    
    return (score_candidate, candidate_trees, parent_id)

# Helper for Forward Search (Hybrid)
def process_forward_candidate(n, packer_params, task_id=0):
    # Fix: Ensure seed is within 32-bit range for numpy
    seed_val = (int(time.time() * 1000000) + os.getpid() + task_id) % (2**32)
    random.seed(seed_val)
    np.random.seed(seed_val)
    
    packer = GreedyPacker(**packer_params)
    
    # Start with one tree
    trees = [ChristmasTree(0, 0, 0)]
    
    for _ in range(n - 1):
        next_tree = packer.place_next_tree(trees, ChristmasTree)
        trees.append(next_tree)
        
    # Optimize
    params = get_sa_params(n)
    trees = optimize_packing(trees, params)
    
    side = get_bounds(trees)
    score = (side ** 2) / n
    return (score, trees, "forward")

# BEAM SEARCH PARAMETERS - OPTIMIZED
BASE_BEAM_WIDTH = CONFIG['dynamic_beam_width']['base_width']
BRANCH_FACTOR = 5     # Reduced from 8 to 5 for speed
n_jobs = multiprocessing.cpu_count()
print(f"Running Optimized Advanced Search (200 -> 1) on {n_jobs} cores.")
print(f"Beam width: {BASE_BEAM_WIDTH}, Branch factor: {BRANCH_FACTOR}")

packer_params = {'n_trials': 200, 'step_size': 0.2, 'fine_step': 0.01}
submission_rows = []
improvements = 0
all_solutions = {}

# Initialize candidates with Lattice for N=200
print("\nInitializing with Lattice Generator for N=200.")
lattice_200 = generate_lattice(200)
# Optimize the lattice initially
print("Optimizing initial lattice...")
lattice_200 = optimize_packing(lattice_200, get_sa_params(200))
current_candidates = [(lattice_200, "lattice")]

# Metrics Tracking
history_n = []
history_improvement = []
history_total_score = []
current_total_score = 0

# Setup Metrics File
os.makedirs('Data', exist_ok=True)
timestamp_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
metrics_file = f'Data/run_metrics_advanced_{timestamp_str}.csv'
with open(metrics_file, 'w') as f:
    f.write('n,score,baseline,improvement,source\n')

# Setup Realtime Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
plt.close(fig) # Don't show yet
plot_display_id = "metrics_plot_advanced"
display(fig, display_id=plot_display_id) # Show initial empty plot

# REVERSE LOOP: 200 down to 1
for n in tqdm(range(200, 0, -1), desc="Processing Reverse"):
    
    print(f"\n--- Processing N={n} ---")

    # Dynamic Beam Width
    current_beam_width = BASE_BEAM_WIDTH
    if CONFIG['dynamic_beam_width']['enabled']:
        # Critical N values: 100, 50, 25, 10
        if n in [100, 50, 25, 10] or n <= 5:
            current_beam_width = int(BASE_BEAM_WIDTH * CONFIG['dynamic_beam_width']['critical_n_multiplier'])
            print(f"  [N={n}] Critical N! Widening beam to {current_beam_width}")

    # Get baseline side for pruning/comparison
    baseline_side = None
    if n in known_solutions:
        baseline_side = get_bounds(known_solutions[n])
    
    # 1. Expand candidates in parallel
    tasks = []
    task_counter = 0
    # Enumerate candidates to assign parent IDs
    for p_idx, (base_trees, _) in enumerate(current_candidates):
        # Identify boundary trees for intelligent removal
        minx, miny, maxx, maxy = unary_union([t.polygon for t in base_trees]).bounds
        boundary_indices = []
        for i, t in enumerate(base_trees):
            tb = t.polygon.bounds
            # Check if touching global bounds
            if (abs(tb[0] - minx) < 1e-2 or abs(tb[1] - miny) < 1e-2 or 
                abs(tb[2] - maxx) < 1e-2 or abs(tb[3] - maxy) < 1e-2):
                boundary_indices.append(i)
        
        # If we have enough boundary trees, pick distinct ones. If not, fill with randoms.
        indices_to_try = boundary_indices[:BRANCH_FACTOR]
        while len(indices_to_try) < BRANCH_FACTOR:
            indices_to_try.append(None) # None triggers random selection inside function
            
        for idx in indices_to_try:
            # Pass p_idx as parent_id
            tasks.append((base_trees, n, packer_params, baseline_side, idx, p_idx, task_counter))
            task_counter += 1
            
    print(f"  [N={n}] Launching {len(tasks)} parallel tasks...")
    
    # Run in parallel
    results = []
    try:
        results = Parallel(n_jobs=n_jobs)(
            delayed(process_beam_candidate_reverse)(t[0], t[1], t[2], t[3], t[4], t[5], t[6]) for t in tasks
        )
    except Exception as e:
        print(f"  [N={n}] ERROR in parallel execution: {e}")
        if n in known_solutions:
            results = [((get_bounds(known_solutions[n])**2)/n, known_solutions[n], "fallback")]
    
    # Hybrid Search: Inject Forward Candidates
    if CONFIG['hybrid_search']['enabled'] and n <= CONFIG['hybrid_search']['switch_n']:
        print(f"  [N={n}] Hybrid Search: Generating forward candidates...")
        # Generate a few forward candidates
        forward_tasks = [(n, packer_params, i) for i in range(3)] # Reduced from 4 to 3
        try:
            forward_results = Parallel(n_jobs=n_jobs)(
                delayed(process_forward_candidate)(t[0], t[1], t[2]) for t in forward_tasks
            )
            results.extend(forward_results)
            print(f"  [N={n}] Added {len(forward_results)} forward candidates.")
        except Exception as e:
            print(f"  [N={n}] Forward search failed: {e}")

    # 2. Sort and select top K (Beam Selection with Diversity)
    results.sort(key=lambda x: x[0])
    if results:
        print(f"  [N={n}] Best candidate score: {results[0][0]:.6f}")
    else:
        print(f"  [N={n}] No results found!")
        continue
    
    # Update current candidates with diversity check
    unique_candidates = []
    seen_scores = set()
    parent_counts = {} # Track how many children from each parent
    
    max_children = CONFIG['beam_diversity']['max_children_per_parent'] if CONFIG['beam_diversity']['enabled'] else 999
    
    for res in results:
        score = round(res[0], 6)
        trees = res[1]
        pid = res[2]
        
        # Diversity Check 1: Score Uniqueness
        if score in seen_scores:
            continue
            
        # Diversity Check 2: Parent Limit
        p_count = parent_counts.get(pid, 0)
        if p_count >= max_children:
            continue
            
        unique_candidates.append((trees, pid))
        seen_scores.add(score)
        parent_counts[pid] = p_count + 1
        
        if len(unique_candidates) >= current_beam_width:
            break
            
    current_candidates = unique_candidates
    
    # The best candidate for this N
    best_score_greedy = results[0][0]
    best_trees_greedy = results[0][1]
    
    # 3. Compare with Baseline - FIXED LOGIC
    best_trees_n = best_trees_greedy
    best_score_n = best_score_greedy
    source = "ReverseGreedy"
    baseline_val = 0
    
    if n in known_solutions:
        known_trees = known_solutions[n]
        # Don't modify the original baseline
        known_trees_copy = [ChristmasTree(t.center_x, t.center_y, t.angle) for t in known_trees]
        known_trees_copy = center_packing(known_trees_copy)
        side_known = get_bounds(known_trees_copy)
        score_known = (side_known ** 2) / n
        baseline_val = score_known
        
        print(f"  [N={n}] Baseline score: {score_known:.6f}, Candidate score: {best_score_greedy:.6f}")

        # Compare (Lower is better)
        if score_known < best_score_greedy - 1e-7:
            # Baseline is better - use it and try to optimize
            best_trees_n = known_trees_copy
            best_score_n = score_known
            source = "Baseline"
            print(f"  [N={n}] Using baseline (better than candidates)")
            
            # Try to optimize baseline
            optimized_known = [ChristmasTree(t.center_x, t.center_y, t.angle) for t in known_trees]
            params = get_sa_params(n)
            params['iterations'] = int(params['iterations'] * 1.5)
            
            optimized_known = optimize_packing(optimized_known, params)
            
            side_opt = get_bounds(optimized_known)
            score_opt = (side_opt ** 2) / n
            
            if score_opt < score_known - 1e-7:
                best_trees_n = optimized_known
                best_score_n = score_opt
                source = "Baseline+Opt"
                improvements += 1
                print(f"  [N={n}] ✓ Optimized baseline: {score_known:.6f} -> {score_opt:.6f} (Δ={score_known-score_opt:.6f})")
            
            # Inject baseline into beam
            current_candidates.append((best_trees_n, "baseline"))
            temp_candidates = []
            for c, pid in current_candidates:
                s = get_bounds(c)
                temp_candidates.append(((s**2)/n, c, pid))
            temp_candidates.sort(key=lambda x: x[0])
            current_candidates = [(x[1], x[2]) for x in temp_candidates[:current_beam_width]]
            
        elif best_score_greedy < score_known - 1e-7:
            # Our candidate beat the baseline
            improvements += 1
            delta = score_known - best_score_greedy
            pct_improvement = (delta / score_known) * 100
            print(f"  [N={n}] ✓ IMPROVED over baseline: {score_known:.6f} -> {best_score_greedy:.6f} (Δ={delta:.6f}, {pct_improvement:.2f}%)")
        else:
            # Scores are essentially equal
            print(f"  [N={n}] Matched baseline (no improvement)")
    else:
        print(f"  [N={n}] No baseline available, using candidate")
            
    # 4. Update State & Check Bounds
    all_solutions[n] = best_trees_n
    
    # Validation - OPTIMIZED (only check periodically)
    if CONFIG['validation']['strict_boundary']:
        for t in best_trees_n:
            if abs(t.center_x) > 100 or abs(t.center_y) > 100:
                print(f"⚠️  WARNING: Tree at N={n} outside bounds: ({t.center_x:.2f}, {t.center_y:.2f})")
    
    # Collision check - OPTIMIZED (only every 10th N and using STRtree for speed)
    if CONFIG['validation']['collision_check'] and n % 10 == 0:
        polys = [t.polygon for t in best_trees_n]
        tree_index = STRtree(polys)
        collision_count = 0
        for i, poly in enumerate(polys):
            candidates = tree_index.query(poly)
            for j in candidates:
                if j > i and polys[i].intersects(polys[j]) and not polys[i].touches(polys[j]):
                    area = polys[i].intersection(polys[j]).area
                    if area > 1e-5:  # Only count significant overlaps
                        collision_count += 1
                        if collision_count == 1:
                            print(f"⚠️  Collision at N={n} between trees {i} and {j} (overlap area={area:.6e})")
        
        if collision_count > 0:
            print(f"  [N={n}] Found {collision_count} collision(s)")
            # If significant collisions found, use baseline instead
            if collision_count > 5 and n in known_solutions:
                print(f"  [N={n}] Too many collisions - reverting to baseline")
                best_trees_n = known_solutions[n]
                best_score_n = (get_bounds(best_trees_n) ** 2) / n
                source = "Baseline_CollisionFallback"

    # 5. Metrics & Plotting
    current_total_score += best_score_n
    imp = max(0, baseline_val - best_score_n) if baseline_val > 0 else 0
    
    print(f"  [N={n}] Cumulative Score: {current_total_score:.6f} | Total Improvements: {improvements}")

    history_n.append(n)
    history_improvement.append(imp)
    history_total_score.append(current_total_score)
    
    # Save metrics
    with open(metrics_file, 'a') as f:
        f.write(f"{n},{best_score_n:.10f},{baseline_val:.10f},{imp:.10f},{source}\n")
        
    # Realtime Plot (Update every 5 steps)
    if n % 5 == 0 or n == 1:
        # Update axes
        ax1.clear()
        ax2.clear()
        
        # Note: history_n is decreasing [200, 199, ...]
        ax1.plot(history_n, history_improvement, 'g-', label='Improvement')
        ax1.set_title(f'Improvement per N (Total: {improvements})')
        ax1.set_xlabel('N')
        ax1.set_ylabel('Score Reduction')
        ax1.invert_xaxis() # Invert X axis to show 200 -> 1 flow
        ax1.grid(True, alpha=0.3)
        
        ax2.plot(history_n, history_total_score, 'b-', label='Total Score')
        ax2.set_title(f'Cumulative Score: {current_total_score:.4f}')
        ax2.set_xlabel('N')
        ax2.set_ylabel('Total Score')
        ax2.invert_xaxis()
        ax2.grid(True, alpha=0.3)
        
        # Update the display
        display(fig, display_id=plot_display_id, update=True)
    
    # 6. Prepare Submission Rows
    for i, tree in enumerate(best_trees_n):
        submission_rows.append([
            f"{n:03d}_{i}", 
            f"s{tree.center_x:.10f}", 
            f"s{tree.center_y:.10f}", 
            f"s{tree.angle:.10f}"
        ])

print(f"\n{'='*60}")
print(f"Processing complete!")
print(f"Total improvements over baseline: {improvements}")
print(f"Final cumulative score: {current_total_score:.6f}")
print(f"{'='*60}")

df_sub = pd.DataFrame(submission_rows, columns=['id', 'x', 'y', 'deg'])
# Sort by ID to ensure 001_... comes first
df_sub.sort_values('id', inplace=True)
df_sub.to_csv('submission.csv', index=False)
print("\n✓ Submission generated: submission.csv")


  [N=190] Best candidate score: 0.168013
  [N=190] Baseline score: 0.367121, Candidate score: 0.168013
  [N=190] ✓ IMPROVED over baseline: 0.367121 -> 0.168013 (Δ=0.199108, 54.23%)
⚠️  Collision at N=190 between trees 0 and 2 (overlap area=1.000000e-02)
  [N=190] Found 810 collision(s)
  [N=190] Too many collisions - reverting to baseline
  [N=190] Cumulative Score: 2.211179 | Total Improvements: 11

--- Processing N=189 ---
  [N=189] Launching 5 parallel tasks...

--- Processing N=189 ---
  [N=189] Launching 5 parallel tasks...
  [N=189] Best candidate score: 0.168902
  [N=189] Baseline score: 0.369011, Candidate score: 0.168902
  [N=189] ✓ IMPROVED over baseline: 0.369011 -> 0.168902 (Δ=0.200109, 54.23%)
  [N=189] Cumulative Score: 2.380081 | Total Improvements: 12

--- Processing N=188 ---
  [N=188] Launching 5 parallel tasks...
  [N=189] Best candidate score: 0.168902
  [N=189] Baseline score: 0.369011, Candidate score: 0.168902
  [N=189] ✓ IMPROVED over baseline: 0.369011 -> 0.168

## 10. Evaluation Helper

Calculate the local score to estimate leaderboard performance.


In [67]:
## 9. Polishing & Refinement
# Run this cell to further optimize the existing submission using the new Numba engine.
# This is much faster than the full search and can squeeze out extra points.

print("Starting Polishing Phase...")
try:
    # Load current best submission
    if os.path.exists('submission.csv'):
        polish_df = pd.read_csv('submission.csv')
        print("Loaded submission.csv for polishing.")
    else:
        polish_df = pd.read_csv('test.csv')
        print("Loaded test.csv for polishing.")

    def parse_s(val):
        return float(str(val).replace('s', ''))
    
    polish_df['x'] = polish_df['x'].apply(parse_s)
    polish_df['y'] = polish_df['y'].apply(parse_s)
    polish_df['deg'] = polish_df['deg'].apply(parse_s)
    
    polish_solutions = {}
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        rows = polish_df[polish_df['id'].str.startswith(prefix)]
        if len(rows) == n:
            trees = []
            for _, row in rows.iterrows():
                trees.append(ChristmasTree(row['x'], row['y'], row['deg']))
            polish_solutions[n] = trees

    total_score_before = 0
    total_score_after = 0
    
    # Polish each N
    for n in tqdm(range(1, 201), desc="Polishing"):
        if n not in polish_solutions: continue
        
        trees = polish_solutions[n]
        side_before = get_bounds(trees)
        score_before = (side_before ** 2) / n
        total_score_before += score_before
        
        # Run Numba Optimization with Bucketed Params
        params = get_sa_params(n).copy()
        
        # Boost iterations for polishing
        # We want a very thorough local search
        params['iterations'] = 200000 if n < 50 else 500000
        
        # Reduce initial temperature for polishing (we are already close to a good solution)
        params['initial_temp'] = params['initial_temp'] * 0.5
        
        optimized_trees = optimize_packing(trees, params)
        
        side_after = get_bounds(optimized_trees)
        score_after = (side_after ** 2) / n
        
        if score_after < score_before - 1e-9:
            polish_solutions[n] = optimized_trees
            total_score_after += score_after
            # print(f"  N={n} Improved: {score_before:.6f} -> {score_after:.6f}")
        else:
            total_score_after += score_before # Keep original
            
    print(f"Polishing Complete.")
    print(f"Score Before: {total_score_before:.6f}")
    print(f"Score After:  {total_score_after:.6f}")
    print(f"Improvement:  {total_score_before - total_score_after:.6f}")
    
    # Save if improved
    if total_score_after < total_score_before:
        new_rows = []
        for n in range(1, 201):
            if n in polish_solutions:
                for i, tree in enumerate(polish_solutions[n]):
                    new_rows.append([
                        f"{n:03d}_{i}", 
                        f"s{tree.center_x:.10f}", 
                        f"s{tree.center_y:.10f}", 
                        f"s{tree.angle:.10f}"
                    ])
        
        df_polished = pd.DataFrame(new_rows, columns=['id', 'x', 'y', 'deg'])
        df_polished.sort_values('id', inplace=True)
        df_polished.to_csv('submission_polished.csv', index=False)
        df_polished.to_csv('submission.csv', index=False) # Overwrite main
        print("Saved polished submission to submission.csv")
        
except Exception as e:
    print(f"Polishing failed: {e}")

Starting Polishing Phase...
Loaded submission.csv for polishing.


Polishing:   0%|          | 0/200 [00:00<?, ?it/s]

Polishing Complete.
Score Before: 44.790548
Score After:  44.737297
Improvement:  0.053251
Saved polished submission to submission.csv


## NEW: Worst-N Refinement Strategy
The scoring metric is Σ(s²/n). Small N with bad packing hurt disproportionately.
This cell identifies the worst performing N values and runs intensive optimization on them.

In [68]:
from optimization_utils import sa_numba_growing
import numpy as np

def refine_worst_n(all_solutions, top_k=30, seeds=20, iterations=100000):
    """Refine worst-performing N values based on Σ(s^2/n) contribution.
    Uses normalized SA params to avoid KeyError and multi-seed Growing Trees.
    """
    print(f"\n{'='*50}\nWORST-N REFINEMENT (top_k={top_k}, seeds={seeds}, iter={iterations})\n{'='*50}")
    if not all_solutions:
        print("No solutions to refine.")
        return all_solutions

    # Build contribution table
    contrib_rows = []
    for n, trees in all_solutions.items():
        side = get_bounds(trees)
        contrib = (side * side) / n
        contrib_rows.append((n, side, contrib))
    arr = np.array(contrib_rows, dtype=np.float64)
    worst_idxs = np.argsort(arr[:, 2])[::-1][:top_k]
    worst_list = arr[worst_idxs, 0].astype(int).tolist()

    print("Worst N selection:")
    for rank, idx in enumerate(worst_idxs, 1):
        n_val = int(arr[idx, 0]); s_val = arr[idx, 1]; c_val = arr[idx, 2]
        print(f"  {rank:2d}. N={n_val:3d} side={s_val:.5f} contrib={c_val:.5f}")

    refined = dict(all_solutions)
    improvements = 0

    for n in tqdm(worst_list, desc="Refining"):
        trees = refined[n]
        base_side = get_bounds(trees)
        xs = np.fromiter((t.center_x for t in trees), dtype=np.float64, count=len(trees))
        ys = np.fromiter((t.center_y for t in trees), dtype=np.float64, count=len(trees))
        angs = np.fromiter((t.angle for t in trees), dtype=np.float64, count=len(trees))

        raw_params = get_sa_params(n)
        p = normalize_sa_params(raw_params)
        T0, Tmin = p['T0'], p['Tmin']
        move_scale, rot_scale = p['move_scale'], p['rot_scale']
        compression = p['compression']
        iter_use = iterations or p['iterations']

        best_side = base_side
        best_bxs, best_bys, best_bangs = xs, ys, angs

        for seed in range(seeds):
            bxs, bys, bangs, side_candidate = sa_numba_growing(
                xs, ys, angs, n,
                iterations=iter_use,
                T0=T0,
                Tmin=Tmin,
                move_scale=move_scale,
                rot_scale=rot_scale,
                seed=seed + 12345,
                compression=compression
            )
            if side_candidate < best_side - 1e-12:
                best_side = side_candidate
                best_bxs, best_bys, best_bangs = bxs.copy(), bys.copy(), bangs.copy()

        if best_side < base_side - 1e-9:
            refined[n] = [ChristmasTree(best_bxs[i], best_bys[i], best_bangs[i]) for i in range(n)]
            improvements += 1
            pct = 100.0 * (base_side - best_side) / base_side
            print(f"  ✓ N={n:3d} improved {base_side:.5f} -> {best_side:.5f} (-{pct:.2f}%)")

    print(f"Refinement complete. Improved {improvements}/{len(worst_list)} targets.")
    return refined

# Example:
# all_solutions = refine_worst_n(all_solutions, top_k=30, seeds=20, iterations=80000)

## NEW: Forward Chain Search
Instead of only using Reverse Beam (200→1), also build solutions forward (1→200).
This helps with tricky N values (primes, etc.) that don't fit well as subsets.

In [69]:
def forward_chain_search(max_n=200, seeds_per_n=3, iterations=30000):
    """Build solutions forward 1..max_n, inserting one tree then optimizing.
    Uses normalized SA params and reduced redundant calculations."""
    print(f"\n{'='*50}\nFORWARD CHAIN SEARCH (max_n={max_n})\n{'='*50}")
    forward = {1: [ChristmasTree(0.0, 0.0, 0.0)]}

    for n in tqdm(range(2, max_n + 1), desc="ForwardChain"):
        prev = forward[n - 1]
        m = len(prev)
        xs = np.fromiter((t.center_x for t in prev), dtype=np.float64, count=m)
        ys = np.fromiter((t.center_y for t in prev), dtype=np.float64, count=m)
        angs = np.fromiter((t.angle for t in prev), dtype=np.float64, count=m)

        minx, maxx = xs.min(), xs.max()
        miny, maxy = ys.min(), ys.max()
        cx, cy = (minx + maxx) / 2.0, (miny + maxy) / 2.0

        candidates = [
            (maxx + 0.6, cy, np.random.rand() * 360.0),
            (cx + np.random.randn() * 0.4, cy + np.random.randn() * 0.4, np.random.rand() * 360.0),
            (minx - 0.5, maxy + 0.5, np.random.rand() * 360.0),
        ]

        raw_params = get_sa_params(n)
        p = normalize_sa_params(raw_params)
        T0, Tmin = p['T0'], p['Tmin']
        move_scale, rot_scale = p['move_scale'], p['rot_scale']
        compression = p['compression']
        iter_use = iterations or p['iterations']

        best_side = float('inf')
        best_sol = None

        for (ix, iy, iang) in candidates:
            xs_new = np.concatenate([xs, [ix]])
            ys_new = np.concatenate([ys, [iy]])
            angs_new = np.concatenate([angs, [iang]])
            for seed in range(seeds_per_n):
                bxs, bys, bangs, side = sa_numba_growing(
                    xs_new, ys_new, angs_new, n,
                    iterations=iter_use,
                    T0=T0, Tmin=Tmin,
                    move_scale=move_scale, rot_scale=rot_scale,
                    seed=seed + n * 1000 + 777,
                    compression=compression
                )
                if side < best_side - 1e-12:
                    best_side = side
                    best_sol = (bxs.copy(), bys.copy(), bangs.copy())

        if best_sol is None:
            forward[n] = prev + [ChristmasTree(cx, cy, 0.0)]
        else:
            bxs, bys, bangs = best_sol
            forward[n] = [ChristmasTree(bxs[i], bys[i], bangs[i]) for i in range(n)]

    print("Forward chain complete.")
    return forward

# forward_solutions = forward_chain_search(max_n=200, seeds_per_n=3, iterations=25000)

## NEW: Solution Ensemble & Merge
Combine results from multiple strategies (Reverse Beam, Forward Chain, Refinement).
For each N, pick the solution with the smallest bounding box.

In [70]:
def merge_solutions(*solution_dicts, labels=None):
    """Merge multiple solution dicts; keep smallest bounding box per N.
    Adds win statistics per source."""
    if labels is None:
        labels = [f"S{i+1}" for i in range(len(solution_dicts))]

    merged = {}
    stats = {lab: {'wins': 0, 'attempts': 0} for lab in labels}

    for n in range(1, 201):
        candidate_rows = []
        for sol, lab in zip(solution_dicts, labels):
            if n in sol:
                trees = sol[n]
                side = get_bounds(trees)
                candidate_rows.append((side, trees, lab))
        if not candidate_rows:
            continue
        # Sort by side ascending
        candidate_rows.sort(key=lambda r: r[0])
        best_side, best_trees, best_lab = candidate_rows[0]
        merged[n] = best_trees
        for side, trees, lab in candidate_rows:
            stats[lab]['attempts'] += 1
        stats[best_lab]['wins'] += 1

    # Report
    print("Merge stats:")
    for lab in labels:
        att = stats[lab]['attempts']; wins = stats[lab]['wins']
        wr = (wins / att * 100.0) if att else 0.0
        print(f"  {lab:15s} wins={wins:3d}/{att:3d} ({wr:5.1f}%)")

    total_score = sum((get_bounds(merged[n])**2)/n for n in merged)
    print(f"Merged total score: {total_score:.6f} over {len(merged)} Ns")
    return merged

def save_solution_to_csv(solutions, filename='submission_ensemble.csv'):
    rows = []
    for n in range(1, 201):
        if n in solutions:
            for i, t in enumerate(solutions[n]):
                rows.append([f"{n:03d}_{i}", f"s{t.center_x:.10f}", f"s{t.center_y:.10f}", f"s{t.angle:.10f}"])
    df = pd.DataFrame(rows, columns=['id','x','y','deg']).sort_values('id')
    df.to_csv(filename, index=False)
    print(f"Saved {filename}")
    return df

## UPDATED: Main Optimization with Growing Trees
This cell shows how to integrate the new `sa_numba_growing` engine into your optimization workflow.
The Growing Trees strategy uses soft physics (90% → 100% scaling) to escape local optima.

In [71]:
def optimize_packing_with_growing_trees(trees, params, seeds=5):
    """Optimizes a list of trees with multi-seed Growing Trees using normalized params."""
    n = len(trees)
    if n == 0:
        return trees
    xs = np.fromiter((t.center_x for t in trees), dtype=np.float64, count=n)
    ys = np.fromiter((t.center_y for t in trees), dtype=np.float64, count=n)
    angs = np.fromiter((t.angle for t in trees), dtype=np.float64, count=n)

    p = normalize_sa_params(params or {})
    T0, Tmin = p['T0'], p['Tmin']
    move_scale, rot_scale = p['move_scale'], p['rot_scale']
    compression = p['compression']
    iterations = p['iterations']

    best_side = float('inf')
    best_state = (xs, ys, angs)

    base_seed = p['base_seed']
    ss = np.random.SeedSequence(base_seed)
    seed_list = ss.spawn(seeds)

    for child in seed_list:
        seed = int(child.generate_state(1)[0] % (2**31 - 1))
        bxs, bys, bangs, side = sa_numba_growing(
            xs, ys, angs, n,
            iterations=iterations,
            T0=T0, Tmin=Tmin,
            move_scale=move_scale, rot_scale=rot_scale,
            seed=seed, compression=compression
        )
        if side < best_side - 1e-12:
            best_side = side
            best_state = (bxs.copy(), bys.copy(), bangs.copy())

    bxs, bys, bangs = best_state
    return [ChristmasTree(bxs[i], bys[i], bangs[i]) for i in range(n)]

# optimized_trees = optimize_packing_with_growing_trees(candidate_trees, normalize_sa_params(get_sa_params(len(candidate_trees))), seeds=6)

In [72]:
# ========== MASTER WORKFLOW (Refactored & Robust) ==========
import time
WORKFLOW_CONFIG = {
    'run_forward_chain': False,
    'run_worst_n_refine': True,
    'worst_n_count': 30,
    'worst_n_seeds': 20,
    'worst_n_iterations': 80000,
    'forward_seeds': 3,
    'forward_iterations': 25000,
}

start_global = time.time()
print(f"{'='*70}\nSANTA 2025 OPTIMIZATION PIPELINE\n{'='*70}")

reverse_solutions = all_solutions.copy() if 'all_solutions' in locals() else {}
if reverse_solutions:
    rev_score = sum((get_bounds(reverse_solutions[n])**2)/n for n in reverse_solutions)
    print(f"Loaded {len(reverse_solutions)} reverse solutions. Score={rev_score:.6f}")
else:
    print("No reverse solutions found; run base search first.")

forward_solutions = {}
if WORKFLOW_CONFIG['run_forward_chain']:
    t0 = time.time()
    forward_solutions = forward_chain_search(
        max_n=200,
        seeds_per_n=WORKFLOW_CONFIG['forward_seeds'],
        iterations=WORKFLOW_CONFIG['forward_iterations']
    )
    fwd_score = sum((get_bounds(forward_solutions[n])**2)/n for n in forward_solutions)
    print(f"Forward chain score={fwd_score:.6f} (time {time.time()-t0:.1f}s)")
else:
    print("Forward chain skipped.")

refined_solutions = reverse_solutions
if WORKFLOW_CONFIG['run_worst_n_refine'] and reverse_solutions:
    t0 = time.time()
    refined_solutions = refine_worst_n(
        reverse_solutions,
        top_k=WORKFLOW_CONFIG['worst_n_count'],
        seeds=WORKFLOW_CONFIG['worst_n_seeds'],
        iterations=WORKFLOW_CONFIG['worst_n_iterations']
    )
    ref_score = sum((get_bounds(refined_solutions[n])**2)/n for n in refined_solutions)
    print(f"Refined score={ref_score:.6f} (time {time.time()-t0:.1f}s)")
else:
    print("Worst-N refinement skipped.")

merge_inputs = [refined_solutions]
merge_labels = ["Refined_Reverse"]
if forward_solutions:
    merge_inputs.append(forward_solutions)
    merge_labels.append("Forward")
final_solutions = merge_solutions(*merge_inputs, labels=merge_labels)
final_score = sum((get_bounds(final_solutions[n])**2)/n for n in final_solutions)
print(f"Final merged score={final_score:.6f}")

save_solution_to_csv(final_solutions, 'submission_ensemble.csv')
save_solution_to_csv(final_solutions, 'submission.csv')
all_solutions = final_solutions
print(f"Pipeline time: {time.time()-start_global:.1f}s")


SANTA 2025 OPTIMIZATION PIPELINE
Loaded 200 reverse solutions. Score=41.403259
Forward chain skipped.

WORST-N REFINEMENT (top_k=30, seeds=20, iter=80000)
Loaded 200 reverse solutions. Score=41.403259
Forward chain skipped.

WORST-N REFINEMENT (top_k=30, seeds=20, iter=80000)
Worst N selection:
   1. N=  1 side=0.81317 contrib=0.66125
   2. N=  2 side=0.94950 contrib=0.45078
   3. N=  3 side=1.14203 contrib=0.43475
   4. N= 17 side=2.45000 contrib=0.35309
   5. N= 14 side=2.20000 contrib=0.34571
   6. N= 16 side=2.32500 contrib=0.33785
   7. N= 18 side=2.45000 contrib=0.33347
   8. N= 10 side=1.82444 contrib=0.33286
   9. N= 13 side=2.07500 contrib=0.33120
  10. N=  4 side=1.15000 contrib=0.33063
  11. N= 12 side=1.96777 contrib=0.32268
  12. N= 15 side=2.20000 contrib=0.32267
  13. N=  9 side=1.70000 contrib=0.32111
  14. N= 19 side=2.45000 contrib=0.31592
  15. N=  8 side=1.57500 contrib=0.31008
  16. N= 11 side=1.84277 contrib=0.30871
  17. N=  7 side=1.45000 contrib=0.30036
  18. N

Refining:   0%|          | 0/30 [00:00<?, ?it/s]

  ✓ N=  1 improved 0.81317 -> 0.73186 (-10.00%)
  ✓ N=  2 improved 0.94950 -> 0.90685 (-4.49%)
  ✓ N=  3 improved 1.14203 -> 1.08706 (-4.81%)
  ✓ N=  3 improved 1.14203 -> 1.08706 (-4.81%)
  ✓ N= 17 improved 2.45000 -> 2.38000 (-2.86%)
  ✓ N= 17 improved 2.45000 -> 2.38000 (-2.86%)
  ✓ N= 14 improved 2.20000 -> 2.13000 (-3.18%)
  ✓ N= 14 improved 2.20000 -> 2.13000 (-3.18%)
  ✓ N= 16 improved 2.32500 -> 2.25500 (-3.01%)
  ✓ N= 16 improved 2.32500 -> 2.25500 (-3.01%)
  ✓ N= 18 improved 2.45000 -> 2.38000 (-2.86%)
  ✓ N= 18 improved 2.45000 -> 2.38000 (-2.86%)
  ✓ N= 10 improved 1.82444 -> 1.63146 (-10.58%)
  ✓ N= 10 improved 1.82444 -> 1.63146 (-10.58%)
  ✓ N= 13 improved 2.07500 -> 2.00500 (-3.37%)
  ✓ N= 13 improved 2.07500 -> 2.00500 (-3.37%)
  ✓ N=  4 improved 1.15000 -> 1.00500 (-12.61%)
  ✓ N=  4 improved 1.15000 -> 1.00500 (-12.61%)
  ✓ N= 12 improved 1.96777 -> 1.88000 (-4.46%)
  ✓ N= 12 improved 1.96777 -> 1.88000 (-4.46%)
  ✓ N= 15 improved 2.20000 -> 2.13000 (-3.18%)
  ✓ N= 1

## Quick Test: Verify Growing Trees Engine

In [73]:
# Test the new Growing Trees engine
print("Testing Growing Trees optimization engine...")
print("-" * 50)

try:
    # Reload the module to get latest changes
    import importlib
    import optimization_utils
    importlib.reload(optimization_utils)
    from optimization_utils import sa_numba_growing
    
    print("✓ Module loaded successfully")
    
    # Create a simple test case with 5 trees
    test_trees = [
        ChristmasTree(0, 0, 0),
        ChristmasTree(1, 0, 45),
        ChristmasTree(0, 1, 90),
        ChristmasTree(-1, 0, 135),
        ChristmasTree(0, -1, 180)
    ]
    
    n = len(test_trees)
    xs = np.array([t.center_x for t in test_trees], dtype=np.float64)
    ys = np.array([t.center_y for t in test_trees], dtype=np.float64)
    angs = np.array([t.angle for t in test_trees], dtype=np.float64)
    
    print(f"✓ Test case created: {n} trees")
    
    # Get initial bounding box
    from optimization_utils import get_poly, get_bbox, calc_side_cached
    cached_bboxes = np.zeros((n, 4), dtype=np.float64)
    for i in range(n):
        px, py = get_poly(xs[i], ys[i], angs[i], 1.0)
        cached_bboxes[i] = get_bbox(px, py)
    
    initial_side = calc_side_cached(cached_bboxes, n)
    print(f"✓ Initial bounding box side: {initial_side:.6f}")
    
    # Run optimization with Growing Trees
    print("\nRunning sa_numba_growing (10k iterations)...")
    bxs, bys, bangs, best_side = sa_numba_growing(
        xs, ys, angs, n,
        iterations=10000,
        T0=1.0,
        Tmin=0.001,
        move_scale=0.5,
        rot_scale=30.0,
        seed=42,
        compression=0.05
    )
    
    print(f"✓ Optimization complete!")
    print(f"  Initial side: {initial_side:.6f}")
    print(f"  Final side:   {best_side:.6f}")
    
    improvement = ((initial_side - best_side) / initial_side) * 100
    print(f"  Improvement:  {improvement:.2f}%")
    
    if best_side < initial_side:
        print("\n✅ SUCCESS: Growing Trees engine is working correctly!")
    else:
        print("\n⚠️  WARNING: No improvement, but engine executed without errors")
    
except Exception as e:
    print(f"\n❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

print("-" * 50)

Testing Growing Trees optimization engine...
--------------------------------------------------
✓ Module loaded successfully
✓ Test case created: 5 trees
✓ Initial bounding box side: 3.150000

Running sa_numba_growing (10k iterations)...
✓ Optimization complete!
  Initial side: 3.150000
  Final side:   3.035000
  Improvement:  3.65%

✅ SUCCESS: Growing Trees engine is working correctly!
--------------------------------------------------


## Performance Benchmark
Benchmark `sa_numba_growing` across varying N and iteration counts to estimate throughput (trees*iterations per second).

In [74]:
import time
from statistics import mean

def benchmark_growing_trees(sample_ns=(5, 20, 50, 100), iterations=5000, repeats=3):
    print(f"Benchmark: iterations={iterations}, repeats={repeats}")
    rows = []
    for n in sample_ns:
        # Random initial layout
        rng = np.random.default_rng(n*17 + 42)
        xs = rng.uniform(-2, 2, size=n)
        ys = rng.uniform(-2, 2, size=n)
        angs = rng.uniform(0, 360, size=n)
        params = get_sa_params(n)
        T0 = params['initial_temp']; Tmin = params['min_temp']
        move_scale = params['move_scale']; rot_scale = params['rot_scale']
        compression = params.get('compression', 0.05)

        times = []
        for r in range(repeats):
            t0 = time.time()
            sa_numba_growing(xs, ys, angs, n, iterations, T0, Tmin, move_scale, rot_scale, seed=r + n*100, compression=compression)
            times.append(time.time() - t0)
        avg = mean(times)
        it_per_sec = iterations / avg
        tree_ops = (iterations * n) / avg
        rows.append((n, avg, it_per_sec, tree_ops))
        print(f"N={n:3d} avg_time={avg:.3f}s  iter/s={it_per_sec:7.0f}  tree-iter/s={tree_ops:8.0f}")
    return rows

# Run benchmark (uncomment to execute):
# benchmark_growing_trees(sample_ns=(10, 40, 80, 160), iterations=8000, repeats=2)

In [ ]:
# Quick verification: Check actual score of submission.csv
print("Verifying submission.csv score...")
try:
    verify_df = pd.read_csv('submission.csv')
    
    def parse_s(val):
        return float(str(val).replace('s', ''))
    
    verify_df['x'] = verify_df['x'].apply(parse_s)
    verify_df['y'] = verify_df['y'].apply(parse_s)
    verify_df['deg'] = verify_df['deg'].apply(parse_s)
    
    verify_score = 0
    verify_configs = {}
    
    for _, row in verify_df.iterrows():
        n_str = row['id'].split('_')[0]
        n = int(n_str)
        if n not in verify_configs:
            verify_configs[n] = []
        verify_configs[n].append(ChristmasTree(row['x'], row['y'], row['deg']))
    
    for n, trees in verify_configs.items():
        if len(trees) != n:
            print(f"⚠️ WARNING: N={n} has {len(trees)} trees (expected {n})")
            continue
        side = get_bounds(trees)
        verify_score += (side ** 2) / n
    
    print(f"\n{'='*60}")
    print(f"Verified Score: {verify_score:.10f}")
    print(f"Configurations: {len(verify_configs)}/200")
    print(f"{'='*60}\n")
    
except FileNotFoundError:
    print("submission.csv not found. Run the main processing cell first.")
except Exception as e:
    print(f"Error verifying: {e}")


In [ ]:
import os
import datetime

# Calculate final score from submission_rows (more reliable than all_solutions dict)
# The all_solutions dict may be incomplete if processing had errors
final_score = 0
score_by_n = {}

print(f"Processing {len(submission_rows)} submission rows...")

for row in submission_rows:
    n_str = row[0].split('_')[0]
    n = int(n_str)
    if n not in score_by_n:
        score_by_n[n] = []
    x = float(row[1].replace('s', ''))
    y = float(row[2].replace('s', ''))
    deg = float(row[3].replace('s', ''))
    score_by_n[n].append(ChristmasTree(x, y, deg))

print(f"Calculating scores for {len(score_by_n)} configurations...")

for n in sorted(score_by_n.keys()):
    trees = score_by_n[n]
    if len(trees) != n:
        print(f"WARNING: N={n} has {len(trees)} trees, expected {n}")
        continue
    side = get_bounds(trees)
    score_n = (side ** 2) / n
    final_score += score_n

print(f"\n{'='*60}")
print(f"Final Score: {final_score:.10f}")
print(f"Configurations found: {len(score_by_n)}/200")
print(f"{'='*60}\n")

# Compare with original
if 'total_test_score' in globals():
    print(f"Original Score: {total_test_score:.10f}")
    if final_score < total_test_score:
        diff = total_test_score - final_score
        print(f"✓ SUCCESS: Score improved by {diff:.10f}!")
        
        # 1. Save copy with detailed name
        os.makedirs('Data', exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        detailed_name = f"Data/submission_score{final_score:.2f}_improved{diff:.2f}_{timestamp}.csv"
        df_sub.to_csv(detailed_name, index=False)
        print(f"Saved backup: {detailed_name}")
        
        # 2. Overwrite test.csv
        df_sub.to_csv('test.csv', index=False)
        print("Overwrote test.csv")
        
        # 3. Submit to Kaggle
        message = f"Improved score {final_score:.6f} (was {total_test_score:.6f})"
        print("Submitting to Kaggle...")
        !kaggle competitions submit -c santa-2025 -f submission.csv -m "{message}"

        # 4. Git Commit and Push
        print("Committing and pushing to Git...")
        !git add .
        !git commit -m "{message}"
        !git push
        
    else:
        improvement = final_score - total_test_score
        print(f"⚠ No improvement (Score change: {improvement:+.10f})")
        print(f"  Current: {final_score:.10f}")
        print(f"  Original: {total_test_score:.10f}")
else:
    print("⚠ Original score not found. Run the cell loading test.csv baseline first.")


Processing 20100 submission rows...
Calculating scores for 200 configurations...
Calculating scores for 200 configurations...

Final Score: 44.7905481598
Configurations found: 200/200

Original Score: 74.6611804330
✓ SUCCESS: Score improved by 29.8706322732!
Saved backup: Data/submission_score44.79_improved29.87_20251124_155633.csv
Overwrote test.csv
Submitting to Kaggle...

Final Score: 44.7905481598
Configurations found: 200/200

Original Score: 74.6611804330
✓ SUCCESS: Score improved by 29.8706322732!
Saved backup: Data/submission_score44.79_improved29.87_20251124_155633.csv
Overwrote test.csv
Submitting to Kaggle...
100%|██████████████████████████████████████| 0.98M/0.98M [00:00<00:00, 2.98MB/s]
100%|██████████████████████████████████████| 0.98M/0.98M [00:00<00:00, 2.98MB/s]
